In [5]:
import os
import pandas as pd
import re
from io import StringIO

# Import the pipeline that calls the LLM
from utils.pipeline import run_pipeline

# Configurable parameter: top_k (must be less than 15)
TOP_K = 1  # Adjust as needed

# Paths
TESTSET_CSV_PATH = "/home/jaf/battery-lifespan-kg/eval/evaluator_question_short.csv"  # Update if needed
BATTERY_FILES_DIR = "/home/jaf/battery-lifespan-kg/resources/testset"
OUTPUT_DIR = "/home/jaf/battery-lifespan-kg/eval"

def save_llm_responses():
    # Read the testset CSV
    df = pd.read_csv(TESTSET_CSV_PATH)
    
    # Identify and sort sample question columns (e.g., SAMPLE_QUESTION_1, SAMPLE_QUESTION_2, ...)
    sample_question_cols = sorted(
        [col for col in df.columns if col.startswith("SAMPLE_QUESTION")],
        key=lambda x: int(x.split('_')[-1])
    )
    
    output_rows = []
    
    # Iterate over each test case
    for idx, row in df.iterrows():
        test_battery_id = row["TEST_BATTERY_ID"]
        test_battery_query_feature = row["TEST_BATTERY_QUERY_FEATURE"]
        battery_file_path = os.path.join(BATTERY_FILES_DIR, f"{test_battery_id}.txt")
        
        # Read the battery file content
        try:
            with open(battery_file_path, "r") as f:
                file_content = f.read()
        except Exception as e:
            print(f"Error reading file for battery {test_battery_id}: {e}")
            continue
        
        # Prepare the row data dictionary with base columns
        row_data = {
            "TEST_BATTERY_ID": test_battery_id,
            "TEST_BATTERY_QUERY_FEATURE": test_battery_query_feature
        }
        
        # For each sample question, obtain the raw LLM response
        for i, question_col in enumerate(sample_question_cols, start=1):
            base_question = str(row[question_col]).strip()
            
            # Append instruction to force the list format with top-K results
            modified_question = (
                f"{base_question} return the top-{TOP_K} results from the most similar first. "
                "Please respond with a comma-separated list of battery IDs only."
            )
            
            # Create a new file-like object for each call (to avoid pointer issues)
            uploaded_file = StringIO(file_content)
            
            try:
                # Run the pipeline with the modified question and the file content
                response = run_pipeline(modified_question, uploaded_file)
                if isinstance(response, str):
                    raw_response = response
                else:
                    raw_response = response.get("result", "")
            except Exception as e:
                raw_response = f"API error: {str(e)}"
                print(f"Error processing battery {test_battery_id} question '{base_question}': {str(e)}")
            
            # Save the original question and its raw response in the row data
            row_data[f"q{i}"] = base_question
            row_data[f"q{i}_raw_response"] = raw_response
        
        output_rows.append(row_data)
    
    # Create a DataFrame from the collected rows
    output_df = pd.DataFrame(output_rows)
    
    # Construct the output CSV path using the input CSV name
    input_csv_name = os.path.basename(TESTSET_CSV_PATH)
    output_csv_path = os.path.join(OUTPUT_DIR, f"llm_response_for_{input_csv_name}.csv")
    
    # Save the DataFrame to CSV
    output_df.to_csv(output_csv_path, index=False)
    print(f"LLM responses saved to {output_csv_path}")

save_llm_responses()


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}